This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Visualizing patterns

The [EBSD](reference.rst#kikuchipy.signals.EBSD) and
[EBSDMasterPattern](reference.rst#kikuchipy.signals.EBSDMasterPattern) signals
have a powerful and versatile
[plot()](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.plot)
method provided by HyperSpy. Its uses are greatly detailed in HyperSpy's
[visualisation user guide](http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html).
This section details example uses specific to EBSD and EBSDMasterPattern
signals.

Let's import the necessary libraries and a Nickel EBSD test data set:

In [ ]:
# exchange inline for qt5 for interactive plotting from the pyqt package
%matplotlib inline

import hyperspy.api as hs
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np


# Use kp.load("data.h5") to load your own data
s = kp.data.nickel_ebsd_large(allow_download=True)  # External download
s

## Navigate in custom map

Correlating results from e.g. crystal and phase structure determination, i.e.
indexing, with experimental patterns can inform their interpretation. When
calling `plot` without any input parameters, the navigator map is a grey scale
image with pixel values corresponding to the sum of all detector intensities
within that pattern:

In [ ]:
s.plot()

However, any
[BaseSignal](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal)
signal with a two-dimensional `signal_shape` corresponding to the scan
`navigation_shape` can be passed in to the `navgiator` parameter in
[plot()](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.plot),
including a virtual image showing diffraction contrast, any quality metric map,
or an orientation map or a phase map.

## Virtual image

A virtual backscatter electron (VBSE) image created from any detector region of
interest with the
[get_virtual_bse_intensity()](reference.rst#kikuchipy.signals.EBSD.get_virtual_bse_intensity)
method or
[get_rgb_image()](reference.rst#kikuchipy.generators.VirtualBSEGenerator.get_rgb_image)
explained in the
[virtual backscatter electron imaging](virtual_backscatter_electron_imaging.rst)
section, can be used as a navigator for a scan `s`:

In [ ]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)
print(vbse_gen)
print(vbse_gen.grid_shape)

In [ ]:
vbse_rgb = vbse_gen.get_rgb_image(r=(4, 1), b=(4, 2), g=(4, 3))
vbse_rgb

In [ ]:
s.plot(navigator=vbse_rgb)

## Any image

Images loaded into a
[Signal2D](http://hyperspy.org/hyperspy-doc/current/api/hyperspy._signals.signal2d.html#hyperspy._signals.signal2d.Signal2D)
can be used as navigators, like a quality metric map like the
[image quality map](feature_maps.ipynb#Image-quality) calculated using
[get_image_quality()](reference.rst#kikuchipy.signals.EBSD.get_image_quality):

In [ ]:
s.remove_static_background()
s.remove_dynamic_background()

In [ ]:
iq = s.get_image_quality()
s_iq = hs.signals.Signal2D(iq)
s.plot(navigator=s_iq)

TODO: Orientation RGB image?

Using colour images (apart from creating RGB virtual BSE images, as shown
above), e.g. an orientation ``om`` or phase map, is a bit more involved:

.. code-block::

    >>> om = plt.imread('/path/to/orientation_map.jpg')
    >>> om_scaled = ske.rescale_intensity(om, out_range=np.uint8)
    >>> s_om = hs.signals.Signal2D(om_scaled)
    >>> s_om
    <Signal2D, title: , dimensions: (149|3, 200)>
    >>> s_om = s_om.transpose(signal_axes=1)
    >>> print(s_om, s_om.data.dtype)
    <Signal1D, title: , dimensions: (200, 149|3)> uint8
    >>> s_om.change_dtype('rgb8')
    >>> s_om
    <Signal2D, title: , dimensions: (|200, 149)> [('R', 'u1'), ('G', 'u1'), ('B', 'u1')]
    >>> s.plot(navigator=s_om)

.. _fig-orientation-map-navigator:

.. figure:: _static/image/visualizing_patterns/om_navigation.jpg
    :align: center
    :width: 100%

    Navigating EBSD patterns (left) in an orientation map ``s_om`` (right).

TODO: Gif? Experimental alongside simulated (sampled from master pattern)?

## Plot multiple scans

HyperSpy provides the function :func:`~hyperspy.drawing.utils.plot_signals` to
plot multiple signals with the same navigator, as explained in the `HyperSpy
user guide
<http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html#plotting-several-signals>`_.
This enables e.g. plotting of experimental and simulated patterns side by side
as a visual inspection of the indexing results:

.. code-block:: python

    >>> import hyperspy.api as hs
    >>> nav_shape = s.axes_manager.navigation_shape[::-1]
    >>> s_sim = kp.load("simulated_patterns.h5", scan_size=nav_shape)
    >>> s_sim
    <EBSD, title: simulated_patterns, dimensions: (200, 149|60, 60)>
    >>> hs.plot.plot_signals([s, s_sim], navigator=s_om)

.. _fig-plot-multiple-scans:

.. figure:: _static/image/visualizing_patterns/plot_multiple_scans.gif
    :align: center
    :width: 100%

    Plotting of experimental and simulated patterns side by side for visual
    inspection, using an :ref:`orientation map as navigator
    <fig-orientation-map-navigator>`.

## Plot master patterns

[EBSDMasterPattern](reference.rst#kikuchipy.signals.EBSDMasterPattern) signals
can be navigated along their energy axis and/or the their northern/southern
hemisphere:

In [ ]:
s_mp = kp.data.nickel_ebsd_master_pattern_small(
    projection="spherical", hemisphere="both"
)  # Only a single energy, 20 keV
s_mp.axes_manager

As can be seen from the axes manager, the master pattern has two navigation
axes, a north and south hemisphere, thus, when plotting, we get a slider as a
navigator when plotting:

In [ ]:
s_mp.plot()